#### Данная тетрадка - предсказания на бейзлайне, т.е. преобразования минимальны, сокращение значений в признаках utm_*, device_* чтобы небыло много ohe-признаков

In [1]:
from warnings import filterwarnings
import pickle
import pandas as pd
import numpy as np
import gc
from datetime import datetime as dt
filterwarnings('ignore')

In [2]:
# %%time
# %run final_projects_merging.ipynb

In [3]:
datatypes = {'session_id': 'string',
 'client_id': 'string',
 'visit_date': 'string',
 'visit_time': 'string',
 'visit_number': 'int64',
 'utm_source': 'string',
 'utm_medium':'string',
 'utm_campaign':'string',
 'utm_adcontent':'string',
 'utm_keyword':'string',
 'device_category':'string',
 'device_os':'string',
 'device_brand':'string',
 'device_model':'string',
 'device_screen_resolution': 'string',
 'device_browser':'string',
 'geo_country':'string',
 'geo_city':'string',
 'target_event': 'int64'}

df_base = pd.read_csv("D:\\Libraries\\Projects\\Skillbox.FinalProject\\data\\stage#1.csv", index_col=False, dtype=datatypes )
df_base = df_base.loc[:, ~df_base.columns.str.contains('Unnamed')]

# ***************************************************************************
#                   ДАННЫЙ КУСОК КОДА ОТСЮДА ПОТОМ УДАЛИТЬ 
df = df_base.copy()
%store df
# ***************************************************************************

#df = df[~df['target_event'].isna()]
# df_ohed = df[cols_for_ohe].copy()
# df_ohed = df_ohed.loc[:200000, :] #500000 строк для 32 ГБ (у меня 32 ГБ),  200000 строк - для 16 ГБ оперативки в системе
# df_ohed.target_event = df_ohed.target_event.astype('int64')
# # df_ohed = pd.get_dummies(df_ohed, columns=['utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 
#                                            'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 
#                                            'geo_country', 'geo_city'],
#                         dummy_na=True)

Stored 'df' (DataFrame)


## План
- убираем дубликаты;
- заполняем пропуски

In [4]:
# def view_occupancy(df):
#     length = len(df)
#     dic = dict()
#     for col in df.columns:
#         percent_filled = (length-df[col].isna().sum())/length
#         dic[col] = percent_filled

#     dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
#     for k, v in dic:
#         print('{}: {:.3f}%, {}'.format(k, v*100, df[k].isna().sum()))

# view_occupancy(df_base)

Удаляем следующие признаки:
- device_os;
- utm_keyword;
- device_model

Здесь сделаю предвариельную подготовку датафрейма: удалю строки с дублированным session_id и target_event=0

In [5]:
# df = df_base.copy()
# %store df

предварительно, заменим пропуски значением 'unknown'

## Отсюда, в отдельных тетрадках буду выполнять различные преобразования. 

#### Первая тетрадка, базовое преобразование

В данной тетрадке в признаках оставим только признаки utm_*, device_* и geo_*. Из преобразований - NaN значения заменим на unknown

In [6]:
# %run FinalProject_base_datapreparation.ipynb
# # print(df.shape)
# # view_occupancy(df)

В этом блокноте в признаках utm_*, device_* и geo_* значения количество которых не превышает 1% заменяю значением 'other', + вычленяю номер месяца

In [7]:
# %run FinalProject_datapreparation_1.ipynb

Здесь для фич visit_date и visit_time вытаскиваю месяц и час, считаю для каждого значения приходится целевых действий. Также, для ОС и для браузера делаю категории

In [8]:
# %%time
# %run FinalProject_datapreparation_2.ipynb
# print(dt.now().strftime("%Y-%m-%d %H:%M:%S"))

Здесь я делаю из категориальных переменных категории: считаю сколько для каждого из категориальных фич будет target_event==1

In [9]:
# %%time
# %run FinalProject_datapreparation_3.ipynb
# print(dt.now().strftime("%Y-%m-%d %H:%M:%S"))

Здесь utm_source делим на категории по количеству target_event'ов == 1, часть OneHotEconde'им, часто тоже категории, только в зависимости от имени, например device_os если iOS или Android, что ..., также поиступаю и с device_browser

In [10]:
# %%time
# %run FinalProject_datapreparation_4.ipynb
# print(dt.now().strftime("%Y-%m-%d %H:%M:%S"))

Здесь попробую высчитать, процент target_event'ов для каждого из категориальных признаков

In [11]:
# %%time
# %run FinalProject_datapreparation_5.ipynb
# print(dt.now().strftime("%Y-%m-%d %H:%M:%S"))

Здесь все utm_* представлены как категории, а также device_category и device_os

In [12]:
# %%time
# %run FinalProject_datapreparation_6.ipynb
# print(dt.now().strftime("%Y-%m-%d %H:%M:%S"))

Тут делаю по примеру Торгашина

In [13]:
%%time
%run FinalProject_datapreparation_7.ipynb
print(dt.now().strftime("%Y-%m-%d %H:%M:%S"))

Stored 'df_ohed' (DataFrame)
2024-03-16 13:52:58
CPU times: total: 15 s
Wall time: 18.3 s


## Получение датафрейма и обучение моделей
здесь возвращаем значение

ниже обучаем модели и смотрим результаты

In [14]:
%%time
# res = res.sample(n=200000).reset_index(drop=True)

print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')

%store -r df_ohed
# df_ohed = df_ohed.sample(n=50000).reset_index(drop=True)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc
x = df_ohed.drop(['target_event'], axis=1)
y = df_ohed['target_event']
del df_ohed
gc.collect()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

#=======================================================================================================================
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(class_weight='balanced', random_state=42)
# clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(x_train, y_train)

proba_test = clf.predict_proba(x_test)
proba_test = proba_test[:, 1]
proba_train = clf.predict_proba(x_train)
proba_train = proba_train[:, 1]

predicted_train = clf.predict(x_train)
predicted_test = clf.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predicted_train)
score_test  = accuracy_score(y_test, predicted_test)
print('DesicionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predicted_test))
# del clf
# gc.collect()
#=====================================================================
print(f'\nRandomForestClassifier:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-1, random_state=42)
forest.fit(x_train, y_train)

proba_test = forest.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = forest.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = forest.predict(x_train)
predict_test  = forest.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
# print('\nRandomForestClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))
# del forest
# gc.collect()
#=====================================================================
print(f'\nExtraTreeClassifier:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.ensemble import ExtraTreesClassifier

extree = ExtraTreesClassifier(class_weight='balanced', random_state=42)
extree.fit(x_train, y_train)

proba_test = extree.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = extree.predict_proba(x_train)
proba_train = proba_train[:, 1]

predict_train = extree.predict(x_train)
predict_test  = extree.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
# print('\nExtraTreesClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

print(f'End cell:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')

Start cell: 2024-03-16 13:52:58
DesicionTreeClassifier:
 Качество на тренировочной выборке: 0.5896
 Качество на тестовой выборке:      0.5892833333333334
 ROc AUC на тренировочной выборке:  0.7119231911746414
 ROC AUC на тестовой выборке:       0.655100354449006
[[34175 24012]
 [  631  1182]]

RandomForestClassifier:   2024-03-16 13:53:01
 Качество на тренировочной выборке: 0.6126142857142857
 Качество на тестовой выборке:      0.6127166666666667
 ROc AUC на тренировочной выборке:  0.7064251310262244
 ROC AUC на тестовой выборке:       0.6544698246465209
[[35634 22553]
 [  684  1129]]

ExtraTreeClassifier:   2024-03-16 13:54:04
 Качество на тренировочной выборке: 0.5896
 Качество на тестовой выборке:      0.58975
 ROc AUC на тренировочной выборке:  0.7119231788504996
 ROC AUC на тестовой выборке:       0.6557397284376064
[[34203 23984]
 [  631  1182]]
End cell:   2024-03-16 13:54:42
CPU times: total: 12min 12s
Wall time: 1min 43s


Здесь осуществим подбор гиперпараметров модели

In [15]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.model_selection import GridSearchCV

params = {
    'class_weight': ['balanced'],
    'n_estimators': list(range(25, 45)),
    'max_depth': list(range(5, 10)),
    'min_samples_leaf': list(range(12, 25)),
    'max_features': ['sqrt', 'log2', None]
}


forest = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=forest,
    param_grid=params,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

forest_tuned = RandomForestClassifier(
    class_weight=     'balanced',
    n_estimators=     best_params['n_estimators'],
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features']
)

forest_tuned.fit(x_train, y_train)
proba_test = forest_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = forest_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = forest_tuned.predict(x_train)
predict_test  = forest_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('\nRandomForestClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-16 13:54:42
Fitting 3 folds for each of 3900 candidates, totalling 11700 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'max_depth': 7, 'max_features': None, 'min_samples_leaf': 18, 'n_estimators': 39}

RandomForestClassifier:
 Качество на тренировочной выборке: 0.6091428571428571
 Качество на тестовой выборке:      0.61325
 ROc AUC на тренировочной выборке:  0.6645696857989669
 ROC AUC на тестовой выборке:       0.6746841030664861
[[35640 22547]
 [  658  1155]]
CPU times: total: 2min 54s
Wall time: 2h 20min 22s


In [17]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# from sklearn.model_selection import GridSearchCV

params = {
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'randoms'],
    'max_depth': list(range(5, 15)),
    'min_samples_leaf': list(range(12, 35)),
    'max_features': ['sqrt', 'log2', None]
}


dtc = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=dtc,
    param_grid=params,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

dtc_tuned = DecisionTreeClassifier(
    class_weight=     'balanced',
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features'],
    criterion=        best_params['criterion'],
    splitter=         best_params['splitter']
)

dtc_tuned.fit(x_train, y_train)
proba_test = dtc_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = dtc_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = dtc_tuned.predict(x_train)
predict_test  = dtc_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('\DecisionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-16 16:15:05
Fitting 3 folds for each of 4140 candidates, totalling 12420 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 9, 'max_features': None, 'min_samples_leaf': 32, 'splitter': 'best'}
\DecisionTreeClassifier:
 Качество на тренировочной выборке: 0.56275
 Качество на тестовой выборке:      0.5675666666666667
 ROc AUC на тренировочной выборке:  0.6695372655950902
 ROC AUC на тестовой выборке:       0.669147595161997
[[32820 25367]
 [  579  1234]]
CPU times: total: 2min 47s
Wall time: 15min 54s


In [ ]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# from sklearn.model_selection import GridSearchCV

params = {
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy', 'log_loss'],
#     'splitter': ['best', 'randoms'],
    'max_depth': list(range(5, 15)),
    'min_samples_leaf': list(range(12, 35)),
    'max_features': ['sqrt', 'log2', None]
}


extree = ExtraTreesClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=extree,
    param_grid=params,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

In [21]:
extree_tuned = ExtraTreesClassifier(
    class_weight=     'balanced',
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features'],
    criterion=        best_params['criterion']
#     splitter=         best_params['splitter']
)

extree_tuned.fit(x_train, y_train)
proba_test = extree_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = extree_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = extree_tuned.predict(x_train)
predict_test  = extree_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('\nDecisionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))


DecisionTreeClassifier:
 Качество на тренировочной выборке: 0.5818071428571429
 Качество на тестовой выборке:      0.5833
 ROc AUC на тренировочной выборке:  0.6675412828286347
 ROC AUC на тестовой выборке:       0.6709886362066892
[[33795 24392]
 [  610  1203]]
